## **Machine Learning - WNBA Playoffs Prediction**
This notebook will focus on the undestanding of the data. We will be using SQLite to store the data due to its scalability & the fact that it's a relational schema.

https://docs.python.org/3/library/sqlite3.html

Import sqlite3 and connect to database file

### **Imports**

In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
import seaborn as sns
import feature_selection as fs
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.svm import SVC
import prep_utils as pu
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import pointbiserialr
from sklearn.feature_selection import RFE

PAST_YEARS = 2
TEST_YEAR = 10
KAGGLE_TEST_YEAR = 11

### **Database Connection Setup**
In this phase we will be creating the database and the tables that will be used to store the data. We also added the competition data to the database.

In [16]:
db = sqlite3.connect("db/ac.db")
db_cur = db.cursor()

[df_awards, df_coaches, df_players_teams, df_players, df_series_post, df_teams_post, df_teams] = pu.db_to_pandas(db)

### **Data Preparation**

***Preparing Teams Dataframe***

+ Removed Irrelevant attributes (arena, etc…)
+ Replaced some features by its success rate; e.g made/attempted
+ Created Playoff Rank




In [17]:
# Transform all possible attributes into percentages. (Made / Attempted) & (Offensive & Defensive Rebound %)
df_new_teams = pu.prepare_teams(df_teams,df_teams_post,PAST_YEARS)

df_new_teams = fs.fs_teams(df_new_teams)

df_new_teams = pu.playoff_rank(df_new_teams,df_teams,PAST_YEARS)
df_team_results = df_new_teams[["year","tmID","confID","playoff","rank","team_playoffs_count","playoff_rank"]]


Dropping divID in Teams...


***Preparing Coaches Dataframe***

+ Regular & Playoff win-rate
+ Coach_Awards
+ Num_Playoff appearances



In [18]:
df_new_coaches = pu.prepare_coaches(df_coaches, df_awards,PAST_YEARS)
df_new_coaches = pu.group_coaches(df_new_coaches)

df_new_coaches.drop("coachID", axis = 1, inplace = True)

df_final_coaches = df_new_coaches.copy()
df_final_coaches.columns = df_final_coaches.columns.str.lower()


Dropping Attribute lgID in Coaches...
Creating attribute coach previous regular season win ratio...
Creating attribute coach playoffs win ratio...
Creating attribute coach playoffs count...
Creating attribute coach awards count...
Dropping attribute post_wins..
Dropping attribute post_losses..
Dropping attribute won..
Dropping attribute lost..

Coaches Null Verification:
year                    0
tmID                    0
coachID                 0
coach_reg_wr            0
coach_po_wr             0
coach_playoffs_count    0
coach_awards            0
dtype: int64


***Preparing Players Dataframe***
+ Created Player_Awards;
+ Replaced some features by its success rate; e.g made/attempted;


In [19]:
df_new_players_teams = pu.prepare_player_teams(df_players_teams,df_awards,PAST_YEARS)

Dropping Attribute lgID in Players_Teams...


***Developed 2 Team Ratings:***
+  One rated the players performance from the previous seasons. The goal is to assess the roster's capacity for consistent performance across a range of seasons, player growth, and the team's general quality and stability.

+ A rating that evaluates whether the team has had a roster of talented players in previous years. The goal of this rating is to assess the team's historical trend of acquiring or keeping high-caliber players as well as to provide a quantitative measure of the team's general quality and stability.

In [20]:

# How the team performed in the previous years
previous_team_ratings, df_new_players_team = pu.final_team_ratings(df_players_teams,df_awards, df_players, df_teams, PAST_YEARS)


# How the players performed in the previous years
previous_team_player_ratings = pu.final_player_team_ratings(df_teams, df_new_players_team, df_players, PAST_YEARS,df_new_players_teams[df_new_players_teams['year'] == KAGGLE_TEST_YEAR])





c:\Users\ricar\Desktop\ac\FEUP-AC\prep_utils.py:772: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_TEST_Kaggle.loc[:, 'PowerRating'] = 0
c:\Users\ricar\Desktop\ac\FEUP-AC\prep_utils.py:741: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_data.sort_values(by='year', inplace=True)
c:\Users\ricar\Desktop\ac\FEUP-AC\prep_utils.py:744: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

#### **Merging all dataframes into one final dataframe**

Although initially we are given a relational schema, in order for the machine learning models work with our data we will be merging the relevant data into a single dataframe. This merge will take into consideration the primary & foreign keys of the relational schema, so all data is correctly inserted where it should.

In [21]:

df_players = df_new_players_teams.copy()
df_players = fs.fs_players(df_players,0.2)
df_players = df_players[df_players['year'] != 1]


df_team_results.columns = df_team_results.columns.str.lower()
previous_team_player_ratings.columns = previous_team_player_ratings.columns.str.lower()


merged_data = pd.merge(df_players, df_team_results, on=['tmid', 'year'], how='left')
merged_data = pd.merge(merged_data, df_final_coaches, on=['tmid', 'year'], how='left')
merged_data = pd.merge(merged_data, previous_team_ratings, on=['tmid', 'year'], how='left')
merged_data = pd.merge(merged_data, previous_team_player_ratings, on=['tmid', 'year'], how='left')


#### Point Bisserial Correlation 
In order to evaluate the relationship between the continuous variables and the binary target (playoff), we will apply a statistical measure designed to account for these kinds of situations. This approach considers the connection between a continuous and a binary variable, offering important new information about that relationship. We can better understand how the continuous attributes affect the playoff result by using the Point Biserial Correlation to find patterns and linkages in the data. By using this analytical method, we want to better understand the underlying dynamics and determinants that determine whether a team makes the playoffs or not given its attributes.

In [22]:
data1_10 = merged_data[merged_data['year'] != 11]
fs.bisserial_corr(data1_10)

playoff_rank: 45.04% correlation
team_players_rating: 39.57% correlation
coach_po_wr: 32.54% correlation
total_assists: 29.99% correlation
coach_reg_wr: 29.64% correlation
total_points: 28.92% correlation
player_awards: 27.79% correlation
team_rating: 27.39% correlation
total_blocks: 26.97% correlation
total_minutes: 26.44% correlation
total_turnovers: 25.48% correlation
coach_playoffs_count: 24.50% correlation
rank: 24.16% correlation
total_gs: 23.08% correlation
total_steals: 22.88% correlation
total_pf: 21.06% correlation
team_playoffs_count: 19.10% correlation
total_drebounds_pct: 14.76% correlation
total_orebounds_pct: 14.76% correlation
total_ft_pct: 13.38% correlation
coach_awards: 12.93% correlation
total_fg_pct: 11.87% correlation
total_dq: 11.23% correlation
total_gp: 8.83% correlation
total_three_pct: 7.22% correlation


The majority of attributes demonstrate a statistically significant correlation, save for **total_ft_pct** and **total_three_pct**. This anomaly may be attributed to their representation as percentages,however the correlation test predominantly accounts for linear relationships hence their values. Despite registering low correlation coefficients, these features remain pertinent, as their relevance is nuanced and contextually contingent, underscoring the importance of considering non-linear dynamics in data interpretation.

#### **Dividing the dataset in both train & test sets**



We'll transform the remaining non-numerical attributes into integers, a step favored by models for optimal performance with numerical inputs. Employing a label encoder, each distinct teamID & confID will be assigned a unique integer. Following this encoding, we'll partition the dataset into training and testing sets, laying the groundwork for effective model evaluation and prediction.

The train & test data is divided in the following format:
- The training data will be all years prior to the year we want to predict
- The test data will be the year we want to predict

In [23]:
label_encoder = LabelEncoder()

merged_data['tmid'] = label_encoder.fit_transform(merged_data['tmid'])
merged_data['confid'] = label_encoder.fit_transform(merged_data['confid'])

x = merged_data.drop('playoff', axis=1)
y = merged_data['playoff']

x_train = merged_data[merged_data['year'].between(0, TEST_YEAR - 1)].drop('playoff', axis=1)
y_train = merged_data[merged_data['year'].between(0, TEST_YEAR - 1)]['playoff']

x_test = merged_data[merged_data['year'] == TEST_YEAR].drop('playoff', axis=1)
y_test = merged_data[merged_data['year'] == TEST_YEAR]['playoff']

### RFE
We will be running Recursive Feature Elimination (RFE) across various models, including **Random Forest**, **Logistic Regression**, **Support Vector Machine**, and **Gradient Boosting**, to discern the optimal set of features for each. This involves systematically evaluating the performance of different feature subsets by testing varying numbers of features. Through the **RFE** process, we aim to identify not only the ideal number of features but also the specific attributes that contribute most significantly to the models' efficacy. By iteratively refining the feature selection, we seek to enhance the models' predictive power and streamline their performance with the most influential and relevant variables.

In [24]:
print("Final Features:")
print(x_train.columns)
min_features = 20

"""rfe_classifiers = {
    'Random Forest': RandomForestClassifier(random_state=42),
    'Logistic Regression': LogisticRegression(random_state=42,max_iter=10000),
    'Support Vector Machine': SVC(random_state=42, kernel='linear'),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
}"""
rfe_classifiers = {
    'Random Forest': RandomForestClassifier(random_state=42),
    'Logistic Regression': LogisticRegression(random_state=42,max_iter=10000),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42)
}


classifiers_features = {}

total_features = len(x_train.columns)

best_model_info = {}


for model_name, model in rfe_classifiers.items():
    print(f"\033[1mModel: {model_name}\033[0m")
    
    results = []

    for i in range(min_features, total_features):
        rfe = RFE(model, n_features_to_select=i)
        rfe.fit(x_train, y_train)
        
        selected_features = set(x_train.columns[rfe.support_])
        selected_features.add("tmid")
        selected_features.add("year")
        selected_features = list(selected_features)

        model.fit(x_train[selected_features], y_train)

        accuracy = model.score(x_test[selected_features], y_test)

        results.append((selected_features, accuracy))
    

    # Sort the results based on accuracy in descending order
    results = sorted(results, key=lambda x: x[1], reverse=True)

    classifiers_features[model_name] = results[0][0]
    best_model_info[model_name] = {'features': results[0][0], 'accuracy': results[0][1], 'model': model}
    
    # Print the results
    for features, accuracy in results[:3]:
        print("Selected Features:", features)
        print("Accuracy:" + str(accuracy) + '\n')

best_model_name = max(best_model_info, key=lambda k: best_model_info[k]['accuracy'])
best_model = best_model_info[best_model_name]['model']
best_features = best_model_info[best_model_name]['features']

print(f"\033[1mBest Model: {best_model_name}\033[0m")
print(f"Best Features: {best_features}")
print(f"Accuracy: {best_model_info[best_model_name]['accuracy']}\n")

Final Features:
Index(['year', 'tmid', 'player_awards', 'total_minutes', 'total_points',
       'total_assists', 'total_steals', 'total_blocks', 'total_turnovers',
       'total_pf', 'total_dq', 'total_gs', 'total_gp', 'total_fg_pct',
       'total_ft_pct', 'total_three_pct', 'total_orebounds_pct',
       'total_drebounds_pct', 'confid', 'rank', 'team_playoffs_count',
       'playoff_rank', 'coach_reg_wr', 'coach_po_wr', 'coach_playoffs_count',
       'coach_awards', 'team_rating', 'team_players_rating'],
      dtype='object')
Model: Random Forest
Selected Features: ['total_points', 'total_blocks', 'coach_po_wr', 'team_rating', 'total_three_pct', 'coach_reg_wr', 'total_gs', 'total_dq', 'year', 'total_pf', 'total_minutes', 'total_fg_pct', 'total_turnovers', 'player_awards', 'total_assists', 'total_ft_pct', 'rank', 'tmid', 'playoff_rank', 'total_steals', 'total_orebounds_pct', 'team_players_rating', 'total_gp', 'total_drebounds_pct']
Accuracy:0.6923076923076923

Selected Features: ['tota

While applying a similar approach, we encountered a deviation when working with K-Nearest Neighbors (KNN). Recursive Feature Elimination (RFE) wasn't conducive to the intricacies of KNN. To address this, we seamlessly pivoted to an alternative strategy, implementing SelectKBest for feature selection tailored to KNN's dynamics. 

In [25]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif
results = []
knn = KNeighborsClassifier()
for i in range(min_features, total_features):
   

    selector = SelectKBest(score_func=mutual_info_classif, k=i)
    selector.fit(x_train, y_train)


    selected_features = set(x_train.columns[selector.get_support()])
    selected_features.add("tmid")
    selected_features.add("year")
    selected_features = list(selected_features)

    knn.fit(x_train[selected_features], y_train)

    accuracy = knn.score(x_test[selected_features], y_test)

    results.append((selected_features, accuracy))

results = sorted(results, key=lambda x: x[1], reverse=True)

classifiers_features["K-Nearest Neighbors"] = results[0][0]

# Print the best 3 results
for features, accuracy in results[:3]:
    print("Selected Features:", features)
    print("Accuracy:" + str(accuracy) + '\n')



Selected Features: ['total_points', 'coach_po_wr', 'coach_awards', 'team_rating', 'total_three_pct', 'team_playoffs_count', 'coach_reg_wr', 'total_gs', 'total_dq', 'year', 'total_fg_pct', 'confid', 'total_assists', 'total_ft_pct', 'rank', 'tmid', 'coach_playoffs_count', 'playoff_rank', 'total_steals', 'total_orebounds_pct', 'team_players_rating', 'total_gp', 'total_drebounds_pct']
Accuracy:0.8461538461538461

Selected Features: ['total_points', 'coach_po_wr', 'coach_awards', 'team_rating', 'total_three_pct', 'team_playoffs_count', 'coach_reg_wr', 'total_gs', 'year', 'confid', 'player_awards', 'total_assists', 'total_ft_pct', 'rank', 'tmid', 'coach_playoffs_count', 'playoff_rank', 'total_steals', 'total_orebounds_pct', 'team_players_rating', 'total_drebounds_pct']
Accuracy:0.7692307692307693

Selected Features: ['total_points', 'total_blocks', 'coach_po_wr', 'coach_awards', 'team_rating', 'total_three_pct', 'team_playoffs_count', 'coach_reg_wr', 'total_gs', 'year', 'confid', 'player_awa

Choosing the model with the most promising results in the **tenth** year prompted us to scrutinize its performance across the entire temporal spectrum. We conducted a comprehensive assessment by testing the model on all available years and calculating the average accuracy. Recognizing that excelling in a single year may not be indicative of overall effectiveness, this method ensures a robust evaluation, considering the model's consistency and adaptability across diverse temporal contexts.

In [35]:
accuracy_scores = []

best_model = best_model_info[best_model_name]['model']
best_features = best_model_info[best_model_name]['features']
print(best_model)
probs = False
for test_year in range(3, 11):  # Testing from year 2 to 10
    x_train = merged_data[merged_data['year'].between(1, test_year - 1)].drop('playoff', axis=1)
    y_train = merged_data[merged_data['year'].between(1, test_year - 1)]['playoff']
    
    x_test = merged_data[merged_data['year'] == test_year].drop('playoff', axis=1)
    y_test = merged_data[merged_data['year'] == test_year]['playoff']
    
    # Training the model
    best_model.fit(x_train[best_features], y_train)
    if(probs):
        # Testing the model
        probabilities = best_model.predict_proba(x_test[best_features])
        x_test['probabilities'] = probabilities.tolist()

        
        top_4_teams = (
            x_test.groupby('confid')
            .apply(lambda x: x.iloc[np.argsort(-np.array(x['probabilities'].tolist())[:, 0])][:4])
            .reset_index(drop=True)['tmid']
            .tolist()
        )

        # Create a list where 1 represents the team with one of the top 4 probabilities in their conference
        final_list = [0 if tmid in top_4_teams else 1 for tmid in x_test['tmid']]


        # Calculating accuracy
        accuracy = accuracy_score(y_test, final_list)
        accuracy_scores.append(accuracy)
    else:
        predictions = best_model.predict(x_test[best_features])
    
        # Calculating accuracy
        accuracy = accuracy_score(y_test, predictions)
        accuracy_scores.append(accuracy)
    print(predictions)
    print(f"Accuracy for testing year {test_year}: {accuracy}")
    
# Calculating average accuracy
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
print(f"\nAverage Accuracy across all years: {average_accuracy}\n\n")


for model_name, model in rfe_classifiers.items():
    print(model_name)
    for test_year in range(3, 11): 
        x_train = merged_data[merged_data['year'].between(1, test_year - 1)].drop('playoff', axis=1)
        y_train = merged_data[merged_data['year'].between(1, test_year - 1)]['playoff']
        
        x_test = merged_data[merged_data['year'] == test_year].drop('playoff', axis=1)
        y_test = merged_data[merged_data['year'] == test_year]['playoff']
        
        # Training the model
        model.fit(x_train[classifiers_features[model_name]], y_train)
        
        predictions = model.predict(x_test[classifiers_features[model_name]])
        
        # Calculating accuracy
        accuracy = accuracy_score(y_test, predictions)
        accuracy_scores.append(accuracy)
        print(f"Accuracy for testing year {test_year}: {accuracy}")

    # Calculating average accuracy
    average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
    
    print(f"\nAverage Accuracy across all years: {average_accuracy}")
    print("\n\n")


LogisticRegression(max_iter=10000, random_state=42)
[1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 0. 1. 0.]
Accuracy for testing year 3: 0.5
[1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1.]
Accuracy for testing year 4: 0.42857142857142855
[0. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0.]
Accuracy for testing year 5: 0.5384615384615384
[0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.]
Accuracy for testing year 6: 0.9230769230769231
[1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 1. 0. 1. 0.]
Accuracy for testing year 7: 0.8571428571428571
[0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0.]
Accuracy for testing year 8: 0.6153846153846154
[0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 0.]
Accuracy for testing year 9: 0.7142857142857143
[1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1.]
Accuracy for testing year 10: 0.8461538461538461

Average Accuracy across all years: 0.6778846153846154


Random Forest
Accuracy for testing year 3: 0.625
Accuracy for testing year 4: 0.42857142857142855
Accuracy for testing year 5: 0.46153846153846156
Accuracy for testing year 6

In [36]:

x_train = merged_data[merged_data['year'].between(1, 10)].drop('playoff', axis=1)
y_train = merged_data[merged_data['year'].between(1, 10)]['playoff']

x_test = merged_data[merged_data['year'] == 11].drop('playoff', axis=1)
y_test = merged_data[merged_data['year'] == 11]['playoff']

print(classifiers_features)
print(best_model)

best_model.fit(x_train[best_features], y_train)
predictions = best_model.predict(x_test[best_features])
print(predictions)


y_t = [1,0,0,1,1,0,1,1,1,1,0,1]

print("11 year Accuray:",accuracy_score(y_t, predictions))


{'Random Forest': ['total_points', 'total_blocks', 'coach_po_wr', 'team_rating', 'total_three_pct', 'coach_reg_wr', 'total_gs', 'total_dq', 'year', 'total_pf', 'total_minutes', 'total_fg_pct', 'total_turnovers', 'player_awards', 'total_assists', 'total_ft_pct', 'rank', 'tmid', 'playoff_rank', 'total_steals', 'total_orebounds_pct', 'team_players_rating', 'total_gp', 'total_drebounds_pct'], 'Logistic Regression': ['total_blocks', 'coach_po_wr', 'coach_awards', 'team_rating', 'total_three_pct', 'team_playoffs_count', 'coach_reg_wr', 'total_gs', 'year', 'total_dq', 'total_pf', 'total_minutes', 'total_fg_pct', 'confid', 'total_turnovers', 'player_awards', 'total_ft_pct', 'rank', 'tmid', 'coach_playoffs_count', 'playoff_rank', 'total_steals', 'total_orebounds_pct', 'team_players_rating', 'total_gp', 'total_drebounds_pct'], 'Gradient Boosting': ['total_points', 'total_blocks', 'team_rating', 'total_three_pct', 'coach_reg_wr', 'total_gs', 'total_dq', 'year', 'total_pf', 'total_minutes', 'total

#### GridSearch
Now that we know the best features for each model, we will use gridsearch to fine tune its parameters.

In [28]:
#best_params = fs.grid_search(classifiers_features,x_train,x_test,y_train,y_test)
#print(best_params)

# Random Forest -> {'bootstrap': False, 'criterion': 'entropy', 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 50}

# Logistic -> {'C': 100, 'fit_intercept': False, 'penalty': 'l1', 'solver': 'liblinear'}

# SVM -> {'C': 10, 'gamma': 'scale', 'kernel': 'linear'}

# Gradient -> {'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 50}

# KNN ->  {'n_neighbors': 10, 'p': 2, 'weights': 'uniform'}
"""
import time

model_params = {'Random Forest': {'random_state':42, 'bootstrap': False, 'criterion': 'entropy', 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 50}, 'Logistic Regression': {'random_state':42, 'C': 100, 'fit_intercept': False, 'penalty': 'l1', 'solver': 'liblinear', 'max_iter': 10000}, 'Support Vector Machine': {'random_state':42, 'C': 10, 'gamma': 'scale', 'kernel': 'linear'}, 'Gradient Boosting': {'random_state':42,'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 50}, 'K-Nearest Neighbors': {'n_neighbors': 10, 'p': 2, 'weights': 'uniform'}}

final_classifiers = {
    'Random Forest': RandomForestClassifier(random_state =42),
    'Logistic Regression': LogisticRegression(random_state =42),
    'Support Vector Machine': SVC(random_state =42),
    'Gradient Boosting': GradientBoostingClassifier(random_state =42),
    'K-Nearest Neighbors': KNeighborsClassifier(),
}

for model_name, model in final_classifiers.items():
    start = time.time()
    model.fit(x_train[classifiers_features[model_name]], y_train)
    y_pred = model.predict(x_test[classifiers_features[model_name]])
    
    accuracy = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred)
    end = time.time()
    print(f'Accuracy for {model_name}: {accuracy}')
    print(f'AUC for {model_name}: {auc}')
    print(end-start)
"""

"\nimport time\n\nmodel_params = {'Random Forest': {'random_state':42, 'bootstrap': False, 'criterion': 'entropy', 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 50}, 'Logistic Regression': {'random_state':42, 'C': 100, 'fit_intercept': False, 'penalty': 'l1', 'solver': 'liblinear', 'max_iter': 10000}, 'Support Vector Machine': {'random_state':42, 'C': 10, 'gamma': 'scale', 'kernel': 'linear'}, 'Gradient Boosting': {'random_state':42,'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 50}, 'K-Nearest Neighbors': {'n_neighbors': 10, 'p': 2, 'weights': 'uniform'}}\n\nfinal_classifiers = {\n    'Random Forest': RandomForestClassifier(random_state =42),\n    'Logistic Regression': LogisticRegression(random_state =42),\n    'Support Vector Machine': SVC(random_state =42),\n    'Gradient Boosting': GradientBoostingClassifier(random_state =42),\n    'K-Nearest Neighbors': KNeighborsClassifier(),\n}\n\nfor model_name, model in fina